# WEEK 10

2024/06/10 - 2024/06/16

# 深度学习 C2_W1

## 1.1 初始化方法的影响

- **零初始化**：
  - 将所有权重初始化为零。
  - 问题：对称性破坏，每个神经元的输出相同，无法学习不同特征。
  
- **随机初始化**：
  - 使用小随机数初始化权重，避免对称性。
  - 适用情况：可以使用均匀分布或正态分布。

- **He 初始化**：
  - 适用于 ReLU 激活函数。
  - 公式： $ W = \sqrt{\frac{2}{n}} \cdot \text{randn}(n, m) $

### 代码

```python
def initialize_parameters_he(layer_dims):
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)           
    
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(2 / layer_dims[l-1])
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters
```

## 1.2 数据集的划分

- **训练集**：用于训练模型，调整权重参数。
- **验证集**：用于调参和模型选择，不参与训练。
- **测试集**：用于评估模型的最终性能，不参与训练和验证。

## 1.3 诊断偏差和方差

- **高偏差**（欠拟合）：
  - 训练误差和验证误差都高。
  - 解决方法：增加模型复杂度，训练更长时间，选择更好的特征。

- **高方差**（过拟合）：
  - 训练误差低，但验证误差高。
  - 解决方法：增加训练数据，使用正则化，选择更简单的模型。

## 1.4 正则化方法

- **L2 正则化**：
  - 在损失函数中加入权重平方和的惩罚项。
  - 公式： $J_{reg} = J + \frac{\lambda}{2m} \sum W^2 $

- **Dropout**：
  - 在训练过程中随机忽略一部分神经元。
  - 优点：减少过拟合，增强模型的泛化能力。
  
### 代码

```python
def compute_cost_with_regularization(A3, Y, parameters, lambd):
    m = Y.shape[1]
    W1 = parameters["W1"]
    W2 = parameters["W2"]
    W3 = parameters["W3"]
    
    cross_entropy_cost = compute_cost(A3, Y)
    
    L2_regularization_cost = (1/m) * (lambd/2) * (np.sum(np.square(W1)) + np.sum(np.square(W2)) + np.sum(np.square(W3)))
    
    cost = cross_entropy_cost + L2_regularization_cost
    
    return cost
```

## 1.5 消失梯度和爆炸梯度

- **消失梯度**：梯度在反向传播过程中逐渐变小，导致权重更新缓慢。
- **爆炸梯度**：梯度在反向传播过程中逐渐变大，导致权重更新过大。
- 解决方法：
  - **He 初始化**：使用较大的初始值。
  - **正则化**：L2 正则化和 Dropout。
  - **梯度裁剪**：限制梯度的最大值。

## 1.6 梯度检查

- **梯度检查**：通过数值方法计算梯度，验证反向传播的正确性。
- 实现步骤：
  1. 计算数值梯度： $ \frac{\partial J}{\partial \theta} \approx \frac{J(\theta + \epsilon) - J(\theta - \epsilon)}{2\epsilon} $
  2. 计算反向传播的梯度。
  3. 比较数值梯度和反向传播的梯度。

### 代码

```python
def gradient_check(parameters, gradients, X, Y, epsilon=1e-7):
    parameters_values, _ = dictionary_to_vector(parameters)
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    for i in range(num_parameters):
        thetaplus = np.copy(parameters_values)
        thetaplus[i][0] = thetaplus[i][0] + epsilon
        J_plus[i] = compute_cost(from_vector_to_parameters(thetaplus), X, Y)
        
        thetaminus = np.copy(parameters_values)
        thetaminus[i][0] = thetaminus[i][0] - epsilon
        J_minus[i] = compute_cost(from_vector_to_parameters(thetaminus), X, Y)
        
        gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 * epsilon)
    
    numerator = np.linalg.norm(grad - gradapprox)
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
    difference = numerator / denominator
    
    return difference
```

## 1.7 初始化方法的应用

- **零初始化**：权重初始化为零，不推荐。
- **随机初始化**：小随机数初始化权重。
- **He 初始化**：推荐用于 ReLU 激活函数的初始化方法。

## 1.8 正则化的应用

- 在模型训练过程中加入正则化项，可以减少过拟合，提高模型的泛化能力。
- 正则化方法包括 L2 正则化和 Dropout 正则化。

# 深度学习 C2_W2

## 2.1 优化方法

- **随机梯度下降（SGD）**：
  - 每次迭代更新一个样本的梯度。
  - 优点：计算效率高。
  - 缺点：收敛速度慢，可能陷入局部最优。

- **动量（Momentum）**：
  - 引入动量项，加速收敛。
  - 公式： $ v = \beta v + (1 - \beta) \nabla J $
  - 更新： $ \theta = \theta - \alpha v $

- **RMSProp**：
  - 使用指数加权平均的方法，对梯度进行缩放。
  - 公式： $ S = \beta S + (1 - \beta) \nabla J^2 $
  - 更新： $ \theta = \theta - \alpha \frac{\nabla J}{\sqrt{S + \epsilon}}$

- **Adam**：
  - 结合动量和 RMSProp，具有自适应学习率。
  - 公式：
    - $ v = \beta_1 v + (1 - \beta_1) \nabla J $
    - $ s = \beta_2 s + (1 - \beta_2) \nabla J^2$
  - 更新：$ \theta = \theta - \alpha \frac{v}{\sqrt{s + \epsilon}}$

### 代码

```python
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    L = len(parameters) // 2
    
    for l in range(1, L + 1):
        v["dW" + str(l)] = beta * v["dW" + str(l)] + (1 - beta) * grads["dW" + str(l)]
        v["db" + str(l)] = beta * v["db" + str(l)] + (1 - beta) * grads["db" + str(l)]
        
        parameters["W" + str(l)] -= learning_rate * v["dW" + str(l)]
        parameters["b" + str(l)] -= learning_rate * v["db" + str(l)]
        
    return parameters, v
```

## 2.2 随机小批量

- **随机小批量（Mini-batch）**：
  - 将训练数据划分为多个小批量，每次迭代使用一个小批量进行梯度更新。
  - 优点：可以加速收敛，提高优化效果。

### 代码

```python
def random_mini_batches(X, Y, mini_batch_size=64, seed=0):
    np.random.seed(seed)
    m = X.shape[1]
    mini_batches = []
    
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1, m))
    
    num_complete_minibatches = math.floor(m / mini_batch_size)
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : (k + 1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch

_size : (k + 1) * mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches
```

## 2.3 学习率衰减

- **学习率衰减**：
  - 随着训练过程的进行，逐渐降低学习率。
  - 益处：在训练初期以较大学习率快速收敛，在训练后期以较小学习率精细调整，防止过大的参数更新。

- 常见的学习率衰减方法：
  - **指数衰减**： $ \alpha_t = \alpha_0 \cdot e^{-\lambda t} $
  - **分段常数衰减**：每经过一定的迭代次数，将学习率乘以一个常数。
  - **1/t 衰减**：$ \alpha_t = \frac{\alpha_0}{  1 + \lambda t} $

### 代码

```python
def update_parameters_with_decay(parameters, grads, learning_rate, decay_rate, iteration):
    L = len(parameters) // 2
    
    learning_rate = learning_rate / (1 + decay_rate * iteration)
    
    for l in range(1, L + 1):
        parameters["W" + str(l)] -= learning_rate * grads["dW" + str(l)]
        parameters["b" + str(l)] -= learning_rate * grads["db" + str(l)]
        
    return parameters
```